# Análisis Exploratorio de Datos (EDA) - Pipeline

Este notebook proporciona un análisis exploratorio completo y automático para cualquier dataset.

**Funcionalidades:**
- Carga automática de datos desde CSV
- Detección automática de tipos de datos
- Análisis estadístico descriptivo
- Detección de valores faltantes y outliers
- Visualizaciones automáticas por tipo de variable
- Correlación entre variables numéricas
- Distribución de variables categóricas
- Resumen ejecutivo del dataset

In [1]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Configurar estilos visuales
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("✓ Librerías importadas correctamente")

✓ Librerías importadas correctamente


## 1. Cargar el Dataset

Carga de datos desde CSV. Por defecto busca `Base_de_datos.csv` en la raíz del proyecto.

In [2]:
import os
import json

# Posibles rutas donde puede estar el dataset
possible_paths = [
    "../../alzheimers_disease_data.csv",
    "../../../alzheimers_disease_data.csv", 
    "../../data/alzheimers_disease_data.csv",
    "../../../data/alzheimers_disease_data.csv",
    "alzheimers_disease_data.csv",
    "./data/alzheimers_disease_data.csv"
]

# Cargar configuración
config_path = "../../config.json"
data_path = None

if os.path.exists(config_path):
    with open(config_path, 'r') as f:
        config = json.load(f)
        configured_path = config.get('data_path', '')
        if configured_path and os.path.exists(configured_path):
            data_path = configured_path

# Si no se encontró en config, buscar en rutas posibles
if data_path is None:
    for path in possible_paths:
        if os.path.exists(path):
            data_path = path
            break

# Si aún no se encuentra, mostrar error informativo
if data_path is None:
    print("❌ No se encontró el archivo del dataset.")
    print("📁 Rutas buscadas:")
    for path in possible_paths:
        print(f"   • {path}")
    print("\n💡 Soluciones:")
    print("   1. Colocar 'alzheimers_disease_data.csv' en la carpeta raíz del proyecto")
    print("   2. Actualizar la ruta en config.json")
    print("   3. Verificar la estructura de carpetas del proyecto")
    raise FileNotFoundError("Dataset no encontrado en ninguna de las rutas esperadas")

# Cargar dataset
df = pd.read_csv(data_path)

print(f"✓ Dataset cargado desde: {data_path}")
print(f"  Dimensiones: {df.shape[0]} filas × {df.shape[1]} columnas")
print(f"\n  Primeras filas del dataset:")
df.head()

✓ Dataset cargado desde: ../../../alzheimers_disease_data.csv
  Dimensiones: 2149 filas × 35 columnas

  Primeras filas del dataset:


,PatientID,Age,Gender,Ethnicity,EducationLevel,BMI,Smoking,AlcoholConsumption,PhysicalActivity,DietQuality,...,MemoryComplaints,BehavioralProblems,ADL,Confusion,Disorientation,PersonalityChanges,DifficultyCompletingTasks,Forgetfulness,Diagnosis,DoctorInCharge
0,4751,73,0,0,2,22.927749,0,13.297218,6.327112,1.347214,...,0,0,1.725883,0,0,0,1,0,0,XXXConfid
1,4752,89,0,0,0,26.827681,0,4.542524,7.619885,0.518767,...,0,0,2.592424,0,0,0,0,1,0,XXXConfid
2,4753,73,0,3,1,17.795882,0,19.555085,7.844988,1.826335,...,0,0,7.119548,0,1,0,1,0,0,XXXConfid
3,4754,74,1,0,1,33.800817,1,12.209266,8.428001,7.435604,...,0,1,6.481226,0,0,0,0,0,0,XXXConfid
4,4755,89,0,0,0,20.716974,0,18.454356,6.310461,0.795498,...,0,0,0.014691,0,0,1,1,0,0,XXXConfid


## 1.5 Clasificación y Tipificación de Variables

Identificación explícita del rol y tipo de cada variable en el dataset.


In [3]:
# CLASIFICACIÓN EXPLÍCITA DE VARIABLES
print("="*80)
print("CLASIFICACIÓN Y TIPIFICACIÓN DE VARIABLES")
print("="*80)

# Identificar variables de ID (irrelevantes para análisis)
id_columns = [col for col in df.columns if any(x in col.lower() for x in ['id', 'index', 'patientid', 'doctorincharge'])]

# Identificar target
target_candidates = ['Diagnosis', 'diagnosis', 'target', 'Target']
target_column = None
for col in target_candidates:
    if col in df.columns:
        target_column = col
        break

# Clasificar variables numéricas
numeric_continuous = []
numeric_discrete = []

for col in df.select_dtypes(include=[np.number]).columns:
    if col in id_columns or col == target_column:
        continue
    
    # Heurística: si tiene menos de 10 valores únicos, probablemente es discreta
    unique_count = df[col].nunique()
    if unique_count < 10:
        numeric_discrete.append(col)
    else:
        numeric_continuous.append(col)

# Clasificar variables categóricas
categorical_nominal = []
categorical_ordinal = []
categorical_binary = []

# Palabras clave que sugieren orden
ordinal_keywords = ['severity', 'stage', 'level', 'grade', 'quality', 'score', 'rating']

for col in df.select_dtypes(include=['object', 'category']).columns:
    if col in id_columns or col == target_column:
        continue
    
    unique_count = df[col].nunique()
    
    # Variables binarias
    if unique_count == 2:
        categorical_binary.append(col)
    # Variables ordinales (basado en palabras clave)
    elif any(keyword in col.lower() for keyword in ordinal_keywords):
        categorical_ordinal.append(col)
    # Variables nominales
    else:
        categorical_nominal.append(col)

# Variables numéricas que son realmente categóricas binarias (0/1)
for col in df.select_dtypes(include=[np.number]).columns:
    if col not in id_columns and col != target_column:
        if df[col].nunique() == 2 and set(df[col].unique()).issubset({0, 1, np.nan}):
            # Mover de numérica a binaria
            if col in numeric_discrete:
                numeric_discrete.remove(col)
            categorical_binary.append(col)

print("\n📊 RESUMEN DE CLASIFICACIÓN:")
print("-"*80)
print(f"Total de variables: {len(df.columns)}")
print(f"\n🎯 Variable Objetivo: {target_column if target_column else 'No identificada'}")
print(f"🔢 Variables de Identificación (a eliminar): {len(id_columns)}")
if id_columns:
    for col in id_columns:
        print(f"   • {col}")

print(f"\n📈 VARIABLES NUMÉRICAS ({len(numeric_continuous) + len(numeric_discrete)}):")
print(f"   • Continuas: {len(numeric_continuous)}")
if numeric_continuous:
    for col in numeric_continuous[:10]:  # Mostrar máximo 10
        print(f"      - {col}")
    if len(numeric_continuous) > 10:
        print(f"      ... y {len(numeric_continuous) - 10} más")

print(f"\n   • Discretas: {len(numeric_discrete)}")
if numeric_discrete:
    for col in numeric_discrete:
        print(f"      - {col}")

print(f"\n📊 VARIABLES CATEGÓRICAS ({len(categorical_binary) + len(categorical_nominal) + len(categorical_ordinal)}):")
print(f"   • Binarias: {len(categorical_binary)}")
if categorical_binary:
    for col in categorical_binary[:10]:
        print(f"      - {col}")
    if len(categorical_binary) > 10:
        print(f"      ... y {len(categorical_binary) - 10} más")

print(f"\n   • Nominales: {len(categorical_nominal)}")
if categorical_nominal:
    for col in categorical_nominal:
        print(f"      - {col}")

print(f"\n   • Ordinales: {len(categorical_ordinal)}")
if categorical_ordinal:
    for col in categorical_ordinal:
        print(f"      - {col}")

# Guardar clasificación para uso posterior
variable_classification = {
    'target': target_column,
    'id_columns': id_columns,
    'numeric_continuous': numeric_continuous,
    'numeric_discrete': numeric_discrete,
    'categorical_binary': categorical_binary,
    'categorical_nominal': categorical_nominal,
    'categorical_ordinal': categorical_ordinal
}

print("\n" + "="*80)
print("✅ Clasificación completada y guardada en 'variable_classification'")
print("="*80)


CLASIFICACIÓN Y TIPIFICACIÓN DE VARIABLES

📊 RESUMEN DE CLASIFICACIÓN:
--------------------------------------------------------------------------------
Total de variables: 35

🎯 Variable Objetivo: Diagnosis
🔢 Variables de Identificación (a eliminar): 3
   • PatientID
   • CholesterolTriglycerides
   • DoctorInCharge

📈 VARIABLES NUMÉRICAS (16):
   • Continuas: 14
      - Age
      - BMI
      - AlcoholConsumption
      - PhysicalActivity
      - DietQuality
      - SleepQuality
      - SystolicBP
      - DiastolicBP
      - CholesterolTotal
      - CholesterolLDL
      ... y 4 más

   • Discretas: 2
      - Ethnicity
      - EducationLevel

📊 VARIABLES CATEGÓRICAS (15):
   • Binarias: 15
      - Gender
      - Smoking
      - FamilyHistoryAlzheimers
      - CardiovascularDisease
      - Diabetes
      - Depression
      - HeadInjury
      - Hypertension
      - MemoryComplaints
      - BehavioralProblems
      ... y 5 más

   • Nominales: 0

   • Ordinales: 0

✅ Clasificación completad

## 2. Información General del Dataset

Inspección de tipos de datos, memoria utilizada y estructura general.

In [4]:
# Crear resumen de información
print("="*80)
print("INFORMACIÓN GENERAL DEL DATASET")
print("="*80)
print(f"\nDimensiones: {df.shape[0]} filas × {df.shape[1]} columnas")
print(f"Memoria utilizada: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB\n")

print("TIPOS DE DATOS:")
print("-" * 80)
dtype_info = pd.DataFrame({
    'Columna': df.columns,
    'Tipo': df.dtypes,
    'No Nulos': df.count(),
    'Nulos': df.isnull().sum(),
    '% Nulos': (df.isnull().sum() / len(df) * 100).round(2)
})
print(dtype_info.to_string(index=False))
print("\n" + "="*80)

INFORMACIÓN GENERAL DEL DATASET

Dimensiones: 2149 filas × 35 columnas
Memoria utilizada: 0.69 MB

TIPOS DE DATOS:
--------------------------------------------------------------------------------
                  Columna    Tipo  No Nulos  Nulos  % Nulos
                PatientID   int64      2149      0      0.0
                      Age   int64      2149      0      0.0
                   Gender   int64      2149      0      0.0
                Ethnicity   int64      2149      0      0.0
           EducationLevel   int64      2149      0      0.0
                      BMI float64      2149      0      0.0
                  Smoking   int64      2149      0      0.0
       AlcoholConsumption float64      2149      0      0.0
         PhysicalActivity float64      2149      0      0.0
              DietQuality float64      2149      0      0.0
             SleepQuality float64      2149      0      0.0
  FamilyHistoryAlzheimers   int64      2149      0      0.0
    CardiovascularDiseas

## 3. Estadísticas Descriptivas

Análisis estadístico automático según el tipo de variable.

In [5]:
# Estadísticas para variables numéricas
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()

print("\n" + "="*80)
print("ESTADÍSTICAS NUMÉRICAS")
print("="*80 + "\n")

if numeric_cols:
    stats = df[numeric_cols].describe().T
    stats['Rango'] = df[numeric_cols].max() - df[numeric_cols].min()
    stats['Asimetría'] = df[numeric_cols].skew()
    stats['Curtosis'] = df[numeric_cols].kurtosis()
    print(stats.round(3))
else:
    print("No hay variables numéricas en el dataset.")

print("\n" + "="*80)
print("ESTADÍSTICAS CATEGÓRICAS")
print("="*80 + "\n")

if categorical_cols:
    for col in categorical_cols:
        print(f"📊 {col}")
        print(f"   Valores únicos: {df[col].nunique()}")
        print(f"   Más frecuente: {df[col].value_counts().index[0]} ({df[col].value_counts().values[0]} veces)")
        print(f"   Distribución:\n{df[col].value_counts()}\n")
else:
    print("No hay variables categóricas en el dataset.")


ESTADÍSTICAS NUMÉRICAS

                            count      mean      std       min       25%  \
PatientID                  2149.0  5825.000  620.507  4751.000  5288.000   
Age                        2149.0    74.909    8.990    60.000    67.000   
Gender                     2149.0     0.506    0.500     0.000     0.000   
Ethnicity                  2149.0     0.698    0.996     0.000     0.000   
EducationLevel             2149.0     1.287    0.905     0.000     1.000   
BMI                        2149.0    27.656    7.217    15.009    21.611   
Smoking                    2149.0     0.289    0.453     0.000     0.000   
AlcoholConsumption         2149.0    10.039    5.758     0.002     5.140   
PhysicalActivity           2149.0     4.920    2.857     0.004     2.571   
DietQuality                2149.0     4.993    2.909     0.009     2.458   
SleepQuality               2149.0     7.051    1.764     4.003     5.483   
FamilyHistoryAlzheimers    2149.0     0.252    0.434     0.000 

## 4. Análisis de Valores Faltantes

Detección y visualización de datos faltantes.

In [6]:
# Análisis de valores faltantes
print("\n" + "="*80)
print("ANÁLISIS DE VALORES FALTANTES")
print("="*80 + "\n")

missing_data = pd.DataFrame({
    'Columna': df.columns,
    'Nulos': df.isnull().sum(),
    '% Nulos': (df.isnull().sum() / len(df) * 100).round(2),
    'Tipo': df.dtypes
})

missing_data = missing_data[missing_data['Nulos'] > 0].sort_values('% Nulos', ascending=False)

if len(missing_data) > 0:
    print("Columnas con valores faltantes:")
    print(missing_data.to_string(index=False))
    
    # Visualizar patrones de valores faltantes
    if len(missing_data) <= 10:
        fig, ax = plt.subplots(figsize=(10, 4))
        missing_data_sorted = missing_data.sort_values('% Nulos', ascending=True)
        ax.barh(missing_data_sorted['Columna'], missing_data_sorted['% Nulos'], color='coral')
        ax.set_xlabel('% de Valores Faltantes')
        ax.set_title('Distribución de Valores Faltantes')
        ax.grid(axis='x', alpha=0.3)
        plt.tight_layout()
        plt.show()
else:
    print("✓ No hay valores faltantes en el dataset")


ANÁLISIS DE VALORES FALTANTES

✓ No hay valores faltantes en el dataset


## 4.5 Unificación de Representaciones de Valores Nulos

Estandarización de distintas representaciones de valores faltantes a un formato único.


In [7]:
# Unificar representaciones de valores nulos
print("\n" + "="*80)
print("UNIFICACIÓN DE VALORES NULOS")
print("="*80 + "\n")

# Valores que representan "nulo" en diferentes formatos
null_representations = ['NA', 'N/A', 'na', 'n/a', 'NULL', 'null', 'None', 'none', 
                        '', ' ', '  ', 'NaN', 'nan', 'missing', 'Missing', '-', '--', '?', 'unknown', 'Unknown']

print("🔍 Buscando representaciones alternativas de valores nulos...")
print(f"   Representaciones buscadas: {len(null_representations)}")

# Analizar cada columna
unification_report = []
total_unified = 0

for col in df.columns:
    if df[col].dtype == 'object':  # Solo en columnas de texto
        # Contar valores antes
        nulls_before = df[col].isnull().sum()
        
        # Reemplazar representaciones alternativas con NaN
        df[col] = df[col].replace(null_representations, np.nan)
        
        # También remover strings que son solo espacios
        df[col] = df[col].apply(lambda x: np.nan if isinstance(x, str) and x.strip() == '' else x)
        
        # Contar valores después
        nulls_after = df[col].isnull().sum()
        
        # Si hubo cambios, reportar
        if nulls_after > nulls_before:
            unified_count = nulls_after - nulls_before
            total_unified += unified_count
            unification_report.append({
                'Columna': col,
                'Nulos Antes': nulls_before,
                'Nulos Después': nulls_after,
                'Unificados': unified_count
            })

if unification_report:
    print(f"\n✅ Se unificaron {total_unified} valores nulos en {len(unification_report)} columna(s):\n")
    
    unification_df = pd.DataFrame(unification_report)
    print(unification_df.to_string(index=False))
    
    print(f"\n📊 Total de valores unificados: {total_unified}")
    print(f"   Porcentaje del dataset: {(total_unified / (df.shape[0] * df.shape[1]) * 100):.3f}%")
else:
    print("\n✓ No se encontraron representaciones alternativas de valores nulos")
    print("  Todos los valores nulos ya están en formato estándar (NaN)")

print("\n💡 Recomendación:")
print("   En el preprocesamiento, asegurar que todas las fuentes de datos")
print("   usen un formato consistente para valores nulos (preferiblemente NaN/NULL)")

print("\n" + "="*80)



UNIFICACIÓN DE VALORES NULOS

🔍 Buscando representaciones alternativas de valores nulos...
   Representaciones buscadas: 20

✓ No se encontraron representaciones alternativas de valores nulos
  Todos los valores nulos ya están en formato estándar (NaN)

💡 Recomendación:
   En el preprocesamiento, asegurar que todas las fuentes de datos
   usen un formato consistente para valores nulos (preferiblemente NaN/NULL)



## 4.6 Identificación y Eliminación de Variables Irrelevantes

Detección de columnas sin valor analítico (IDs, columnas constantes, alta cardinalidad sin información).


In [12]:
# Identificar y eliminar variables irrelevantes
print("\n" + "="*80)
print("IDENTIFICACIÓN DE VARIABLES IRRELEVANTES")
print("="*80 + "\n")

irrelevant_columns = []
irrelevant_reasons = {}

# 1. Columnas de identificación (sin valor predictivo)
# Solo identificar por palabras clave explícitas en el nombre
id_keywords = ['id', 'index', 'patient', 'doctor', 'uid', 'key', 'code']
for col in df.columns:
    if any(keyword in col.lower() for keyword in id_keywords):
        if col not in irrelevant_columns:
            irrelevant_columns.append(col)
            irrelevant_reasons[col] = "Columna de identificación (sin valor predictivo)"

# 2. Columnas con un solo valor único (constantes)
for col in df.columns:
    if df[col].nunique() == 1:
        if col not in irrelevant_columns:
            irrelevant_columns.append(col)
            irrelevant_reasons[col] = f"Columna constante (1 único valor: {df[col].unique()[0]})"

# 3. Columnas con casi todos valores nulos (>95%)
null_threshold = 0.95
for col in df.columns:
    null_pct = df[col].isnull().sum() / len(df)
    if null_pct > null_threshold:
        if col not in irrelevant_columns:
            irrelevant_columns.append(col)
            irrelevant_reasons[col] = f"Exceso de nulos ({null_pct*100:.1f}% faltantes)"

# 4. Columnas categóricas con cardinalidad extremadamente alta (>90% valores únicos)
# Estas podrían ser texto libre o IDs enmascarados
for col in df.select_dtypes(include=['object']).columns:
    if col not in irrelevant_columns:
        unique_ratio = df[col].nunique() / len(df)
        if unique_ratio > 0.9:
            irrelevant_columns.append(col)
            irrelevant_reasons[col] = f"Cardinalidad muy alta - Posible ID/Texto libre ({df[col].nunique()} valores únicos, {unique_ratio*100:.1f}% del dataset)"

print(f"🔍 VARIABLES IRRELEVANTES IDENTIFICADAS: {len(irrelevant_columns)}")
print("-"*80)

if irrelevant_columns:
    for col in irrelevant_columns:
        print(f"\n❌ {col}")
        print(f"   Razón: {irrelevant_reasons[col]}")
        print(f"   Tipo: {df[col].dtype}")
        print(f"   Valores únicos: {df[col].nunique()}")
        print(f"   Valores nulos: {df[col].isnull().sum()} ({df[col].isnull().sum()/len(df)*100:.1f}%)")
    
    # Crear dataframe limpio (sin eliminar del original para preservar análisis)
    df_clean = df.drop(columns=irrelevant_columns)
    
    print("\n" + "="*80)
    print(f"✅ Se identificaron {len(irrelevant_columns)} variable(s) irrelevante(s)")
    print(f"📊 Dataset original: {df.shape[0]} filas × {df.shape[1]} columnas")
    print(f"📊 Dataset limpio: {df_clean.shape[0]} filas × {df_clean.shape[1]} columnas")
    print(f"   Columnas eliminadas: {len(irrelevant_columns)}")
    
    print("\n💡 Recomendación:")
    print("   Estas columnas deben eliminarse en el preprocesamiento de datos")
    print("   antes de construir modelos de Machine Learning.")
    
    # Guardar lista para uso posterior
    columns_to_remove = irrelevant_columns
    
else:
    print("\n✓ No se encontraron variables irrelevantes en el dataset")
    print("  Todas las columnas parecen tener valor analítico potencial")
    df_clean = df.copy()
    columns_to_remove = []

print("\n" + "="*80)



IDENTIFICACIÓN DE VARIABLES IRRELEVANTES

🔍 VARIABLES IRRELEVANTES IDENTIFICADAS: 3
--------------------------------------------------------------------------------

❌ PatientID
   Razón: Columna de identificación (sin valor predictivo)
   Tipo: int64
   Valores únicos: 2149
   Valores nulos: 0 (0.0%)

❌ CholesterolTriglycerides
   Razón: Columna de identificación (sin valor predictivo)
   Tipo: float64
   Valores únicos: 2149
   Valores nulos: 0 (0.0%)

❌ DoctorInCharge
   Razón: Columna de identificación (sin valor predictivo)
   Tipo: object
   Valores únicos: 1
   Valores nulos: 0 (0.0%)

✅ Se identificaron 3 variable(s) irrelevante(s)
📊 Dataset original: 2149 filas × 35 columnas
📊 Dataset limpio: 2149 filas × 32 columnas
   Columnas eliminadas: 3

💡 Recomendación:
   Estas columnas deben eliminarse en el preprocesamiento de datos
   antes de construir modelos de Machine Learning.



## 4.7 Conversión y Corrección de Tipos de Datos

Detección y corrección de columnas con tipos de datos incorrectos.


In [13]:
# Detección y corrección de tipos de datos incorrectos
print("\n" + "="*80)
print("CONVERSIÓN Y CORRECCIÓN DE TIPOS DE DATOS")
print("="*80 + "\n")

type_conversions = []

# Trabajar con df_clean si existe, sino con df
df_to_fix = df_clean.copy() if 'df_clean' in locals() else df.copy()

print("🔍 Analizando tipos de datos actuales...\n")

# 1. Detectar números almacenados como strings
for col in df_to_fix.select_dtypes(include=['object']).columns:
    # Intentar convertir a numérico
    try:
        # Verificar si la columna parece numérica
        sample = df_to_fix[col].dropna().head(100)
        
        # Intentar conversión
        converted = pd.to_numeric(sample, errors='coerce')
        
        # Si >80% se convirtió exitosamente, probablemente es numérica
        success_rate = converted.notna().sum() / len(sample)
        
        if success_rate > 0.8:
            df_to_fix[col] = pd.to_numeric(df_to_fix[col], errors='coerce')
            type_conversions.append({
                'Columna': col,
                'Tipo Anterior': 'object',
                'Tipo Nuevo': 'numeric',
                'Razón': f'Columna numérica almacenada como texto ({success_rate*100:.0f}% conversión exitosa)'
            })
    except:
        pass

# 2. Detectar variables binarias que deberían ser categóricas
for col in df_to_fix.select_dtypes(include=[np.number]).columns:
    unique_values = df_to_fix[col].dropna().unique()
    
    if len(unique_values) == 2:
        # Verificar si son 0/1 o similares
        if set(unique_values).issubset({0, 1, 0.0, 1.0}):
            df_to_fix[col] = df_to_fix[col].astype('category')
            type_conversions.append({
                'Columna': col,
                'Tipo Anterior': 'numeric',
                'Tipo Nuevo': 'category',
                'Razón': 'Variable binaria (0/1) - mejor como categórica'
            })

# 3. Detectar variables categóricas con pocos valores únicos
for col in df_to_fix.select_dtypes(include=[np.number]).columns:
    if col not in [c['Columna'] for c in type_conversions]:  # Si no fue ya convertida
        unique_count = df_to_fix[col].nunique()
        
        # Si tiene menos de 10 valores únicos y parecen ser códigos
        if unique_count < 10 and unique_count > 2:
            # Verificar si son enteros consecutivos o códigos
            unique_values = sorted(df_to_fix[col].dropna().unique())
            
            # Si son enteros pequeños, probablemente categórica
            if all(isinstance(x, (int, np.integer)) or x.is_integer() for x in unique_values):
                if max(unique_values) < 20:  # Códigos pequeños
                    df_to_fix[col] = df_to_fix[col].astype('category')
                    type_conversions.append({
                        'Columna': col,
                        'Tipo Anterior': 'numeric',
                        'Tipo Nuevo': 'category',
                        'Razón': f'Variable discreta con {unique_count} valores únicos - posiblemente categórica'
                    })

# 4. Optimizar tipos numéricos (downcast para ahorrar memoria)
for col in df_to_fix.select_dtypes(include=['int64', 'int32']).columns:
    if col not in [c['Columna'] for c in type_conversions]:
        # Intentar downcast a int más pequeño
        original_dtype = df_to_fix[col].dtype
        df_to_fix[col] = pd.to_numeric(df_to_fix[col], downcast='integer')
        
        if df_to_fix[col].dtype != original_dtype:
            type_conversions.append({
                'Columna': col,
                'Tipo Anterior': str(original_dtype),
                'Tipo Nuevo': str(df_to_fix[col].dtype),
                'Razón': 'Optimización de memoria (downcast)'
            })

for col in df_to_fix.select_dtypes(include=['float64', 'float32']).columns:
    if col not in [c['Columna'] for c in type_conversions]:
        # Intentar downcast a float más pequeño
        original_dtype = df_to_fix[col].dtype
        df_to_fix[col] = pd.to_numeric(df_to_fix[col], downcast='float')
        
        if df_to_fix[col].dtype != original_dtype:
            type_conversions.append({
                'Columna': col,
                'Tipo Anterior': str(original_dtype),
                'Tipo Nuevo': str(df_to_fix[col].dtype),
                'Razón': 'Optimización de memoria (downcast)'
            })

print(f"🔄 CONVERSIONES DE TIPOS REALIZADAS: {len(type_conversions)}")
print("-"*80)

if type_conversions:
    conversions_df = pd.DataFrame(type_conversions)
    print(conversions_df.to_string(index=False))
    
    # Calcular ahorro de memoria
    memory_before = df.memory_usage(deep=True).sum() / 1024**2
    memory_after = df_to_fix.memory_usage(deep=True).sum() / 1024**2
    memory_saved = memory_before - memory_after
    
    print(f"\n📊 IMPACTO EN MEMORIA:")
    print(f"   Memoria antes: {memory_before:.2f} MB")
    print(f"   Memoria después: {memory_after:.2f} MB")
    print(f"   Ahorro: {memory_saved:.2f} MB ({(memory_saved/memory_before*100):.1f}%)")
    
    # Actualizar dataframe limpio
    df_clean = df_to_fix
    
else:
    print("\n✓ Todos los tipos de datos son correctos")
    print("  No se requieren conversiones")

print("\n💡 Recomendación:")
print("   Aplicar estas conversiones al inicio del pipeline de preprocesamiento")
print("   para asegurar consistencia en el análisis y modelado.")

print("\n" + "="*80)



CONVERSIÓN Y CORRECCIÓN DE TIPOS DE DATOS

🔍 Analizando tipos de datos actuales...

🔄 CONVERSIONES DE TIPOS REALIZADAS: 32
--------------------------------------------------------------------------------
                  Columna Tipo Anterior Tipo Nuevo                                                            Razón
                   Gender       numeric   category                   Variable binaria (0/1) - mejor como categórica
                  Smoking       numeric   category                   Variable binaria (0/1) - mejor como categórica
  FamilyHistoryAlzheimers       numeric   category                   Variable binaria (0/1) - mejor como categórica
    CardiovascularDisease       numeric   category                   Variable binaria (0/1) - mejor como categórica
                 Diabetes       numeric   category                   Variable binaria (0/1) - mejor como categórica
               Depression       numeric   category                   Variable binaria (0/1) - mejor

## 4.8 Detección y Corrección de Inconsistencias en Datos

Identificación de duplicados, espacios, inconsistencias de formato y valores imposibles.


In [ ]:
# Detección y corrección de inconsistencias
print("\n" + "="*80)
print("DETECCIÓN Y CORRECCIÓN DE INCONSISTENCIAS")
print("="*80 + "\n")

inconsistencies_found = []

# Trabajar con df_clean
df_to_check = df_clean.copy() if 'df_clean' in locals() else df.copy()

# 1. DUPLICADOS
print("1️⃣ ANÁLISIS DE DUPLICADOS:")
print("-"*80)
n_duplicates = df_to_check.duplicated().sum()
print(f"   Filas duplicadas (completas): {n_duplicates}")

if n_duplicates > 0:
    print(f"   Porcentaje: {(n_duplicates/len(df_to_check)*100):.2f}%")
    inconsistencies_found.append(f"Duplicados: {n_duplicates} filas")
    
    # Eliminar duplicados
    df_to_check = df_to_check.drop_duplicates()
    print(f"   ✅ Duplicados eliminados. Filas restantes: {len(df_to_check)}")
else:
    print("   ✓ No hay filas duplicadas")

# 2. ESPACIOS EN BLANCO EN STRINGS
print("\n2️⃣ ANÁLISIS DE ESPACIOS EN STRINGS:")
print("-"*80)
string_issues = []

for col in df_to_check.select_dtypes(include=['object']).columns:
    # Detectar espacios al inicio/final
    if df_to_check[col].dtype == 'object':
        has_leading_spaces = df_to_check[col].apply(lambda x: isinstance(x, str) and x != x.strip()).sum()
        
        if has_leading_spaces > 0:
            string_issues.append(f"{col}: {has_leading_spaces} valores con espacios")
            # Corregir
            df_to_check[col] = df_to_check[col].apply(lambda x: x.strip() if isinstance(x, str) else x)

if string_issues:
    print(f"   ⚠️ Se encontraron {len(string_issues)} columna(s) con espacios:")
    for issue in string_issues:
        print(f"      • {issue}")
    print(f"   ✅ Espacios eliminados")
    inconsistencies_found.extend(string_issues)
else:
    print("   ✓ No hay problemas de espacios en strings")

# 3. INCONSISTENCIAS DE MAYÚSCULAS/MINÚSCULAS
print("\n3️⃣ ANÁLISIS DE INCONSISTENCIAS DE FORMATO:")
print("-"*80)
case_issues = []

for col in df_to_check.select_dtypes(include=['object']).columns:
    unique_values = df_to_check[col].dropna().unique()
    
    if len(unique_values) < 100:  # Solo para columnas categóricas
        # Verificar si hay valores que son iguales excepto por mayúsculas
        lower_values = [str(v).lower() for v in unique_values]
        
        if len(lower_values) != len(set(lower_values)):
            # Hay duplicados en minúsculas
            case_issues.append(col)
            
            # Estandarizar a título (capitalizar primera letra)
            df_to_check[col] = df_to_check[col].apply(lambda x: str(x).title() if pd.notna(x) else x)

if case_issues:
    print(f"   ⚠️ Se encontraron {len(case_issues)} columna(s) con inconsistencias:")
    for col in case_issues:
        print(f"      • {col}")
    print(f"   ✅ Formato estandarizado (Title Case)")
    inconsistencies_found.append(f"Inconsistencias de formato: {len(case_issues)} columnas")
else:
    print("   ✓ No hay inconsistencias de formato en variables categóricas")


# 4. RELACIONES LÓGICAS IMPOSIBLES
print("\n4️⃣ ANÁLISIS DE RELACIONES LÓGICAS:")
print("-"*80)
logical_issues = []

# Ejemplo: Systolic BP debe ser mayor que Diastolic BP
if 'SystolicBP' in df_to_check.columns and 'DiastolicBP' in df_to_check.columns:
    invalid_bp = (df_to_check['SystolicBP'] <= df_to_check['DiastolicBP']).sum()
    if invalid_bp > 0:
        logical_issues.append(f"Presión sistólica ≤ diastólica: {invalid_bp} casos")
        print(f"   ⚠️ {invalid_bp} casos donde Systolic BP ≤ Diastolic BP (imposible)")

# Ejemplo: LDL + HDL no puede ser mayor que Colesterol Total
if all(col in df_to_check.columns for col in ['CholesterolTotal', 'CholesterolLDL', 'CholesterolHDL']):
    invalid_chol = ((df_to_check['CholesterolLDL'] + df_to_check['CholesterolHDL']) > df_to_check['CholesterolTotal'] * 1.2).sum()
    if invalid_chol > 0:
        logical_issues.append(f"LDL+HDL > Total colesterol: {invalid_chol} casos")
        print(f"   ⚠️ {invalid_chol} casos donde LDL+HDL > Colesterol Total (inconsistente)")

if logical_issues:
    inconsistencies_found.extend(logical_issues)
    print(f"\n   💡 Se identificaron {len(logical_issues)} tipo(s) de inconsistencias lógicas")
else:
    print("   ✓ No se detectaron inconsistencias lógicas entre variables")

# RESUMEN
print("\n" + "="*80)
print("📊 RESUMEN DE INCONSISTENCIAS:")
print("="*80)

if inconsistencies_found:
    print(f"\nTotal de problemas identificados: {len(inconsistencies_found)}\n")
    for idx, issue in enumerate(inconsistencies_found, 1):
        print(f"{idx}. {issue}")
    
    print(f"\n✅ Dataset corregido guardado como 'df_clean'")
    print(f"   Filas originales: {len(df)}")
    print(f"   Filas después de limpieza: {len(df_to_check)}")
    print(f"   Filas eliminadas: {len(df) - len(df_to_check)}")
    
    # Actualizar df_clean
    df_clean = df_to_check
else:
    print("\n✓ No se detectaron inconsistencias significativas")
    print("  El dataset es consistente y está listo para análisis")

print("\n" + "="*80)



DETECCIÓN Y CORRECCIÓN DE INCONSISTENCIAS

1️⃣ ANÁLISIS DE DUPLICADOS:
--------------------------------------------------------------------------------
   Filas duplicadas (completas): 0
   ✓ No hay filas duplicadas

2️⃣ ANÁLISIS DE ESPACIOS EN STRINGS:
--------------------------------------------------------------------------------
   ✓ No hay problemas de espacios en strings

3️⃣ ANÁLISIS DE INCONSISTENCIAS DE FORMATO:
--------------------------------------------------------------------------------
   ✓ No hay inconsistencias de formato en variables categóricas

5️⃣ ANÁLISIS DE RELACIONES LÓGICAS:
--------------------------------------------------------------------------------
   ⚠️ 186 casos donde Systolic BP ≤ Diastolic BP (imposible)
   ⚠️ 238 casos donde LDL+HDL > Colesterol Total (inconsistente)

   💡 Se identificaron 2 tipo(s) de inconsistencias lógicas

📊 RESUMEN DE INCONSISTENCIAS:

Total de problemas identificados: 2

1. Presión sistólica ≤ diastólica: 186 casos
2. LDL+HDL 

## 4.9 Estadísticas Descriptivas Después de Limpieza

Comparación de estadísticas antes y después de ajustar tipos de datos y corregir inconsistencias.


In [ ]:
# Ejecutar describe() después de limpieza y ajustes de tipos
print("\n" + "="*80)
print("ESTADÍSTICAS DESCRIPTIVAS - DESPUÉS DE LIMPIEZA")
print("="*80 + "\n")

# Usar df_clean si existe
df_final = df_clean if 'df_clean' in locals() else df

# Separar variables numéricas y categóricas
numeric_cols_clean = df_final.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols_clean = df_final.select_dtypes(include=['object', 'category']).columns.tolist()

print("📊 COMPARACIÓN: ANTES vs DESPUÉS DE LIMPIEZA")
print("-"*80)
print(f"Dataset Original:")
print(f"   • Filas: {df.shape[0]}")
print(f"   • Columnas: {df.shape[1]}")
print(f"   • Variables numéricas: {len(numeric_cols)}")
print(f"   • Variables categóricas: {len(categorical_cols)}")
print(f"   • Memoria: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

print(f"\nDataset Limpio:")
print(f"   • Filas: {df_final.shape[0]} ({df.shape[0] - df_final.shape[0]} eliminadas)")
print(f"   • Columnas: {df_final.shape[1]} ({df.shape[1] - df_final.shape[1]} eliminadas)")
print(f"   • Variables numéricas: {len(numeric_cols_clean)}")
print(f"   • Variables categóricas: {len(categorical_cols_clean)}")
print(f"   • Memoria: {df_final.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Estadísticas descriptivas numéricas
if numeric_cols_clean:
    print("\n" + "="*80)
    print("ESTADÍSTICAS NUMÉRICAS (DESPUÉS DE LIMPIEZA)")
    print("="*80 + "\n")
    
    stats_clean = df_final[numeric_cols_clean].describe().T
    stats_clean['Nulos'] = df_final[numeric_cols_clean].isnull().sum()
    stats_clean['% Nulos'] = (df_final[numeric_cols_clean].isnull().sum() / len(df_final) * 100).round(2)
    stats_clean['Rango'] = df_final[numeric_cols_clean].max() - df_final[numeric_cols_clean].min()
    stats_clean['Asimetría'] = df_final[numeric_cols_clean].skew()
    stats_clean['Curtosis'] = df_final[numeric_cols_clean].kurtosis()
    
    print(stats_clean.round(3))

# Estadísticas descriptivas categóricas
if categorical_cols_clean:
    print("\n" + "="*80)
    print("ESTADÍSTICAS CATEGÓRICAS (DESPUÉS DE LIMPIEZA)")
    print("="*80 + "\n")
    
    for col in categorical_cols_clean:
        print(f"📊 {col}")
        print(f"   Tipo: {df_final[col].dtype}")
        print(f"   Valores únicos: {df_final[col].nunique()}")
        print(f"   Valores nulos: {df_final[col].isnull().sum()} ({df_final[col].isnull().sum()/len(df_final)*100:.1f}%)")
        
        if df_final[col].nunique() <= 10:
            print(f"   Distribución:")
            value_dist = df_final[col].value_counts()
            for val, count in value_dist.items():
                print(f"      • {val}: {count} ({count/len(df_final)*100:.1f}%)")
        else:
            print(f"   Top 5 valores más frecuentes:")
            top_values = df_final[col].value_counts().head(5)
            for val, count in top_values.items():
                print(f"      • {val}: {count} ({count/len(df_final)*100:.1f}%)")
        print()

# Resumen de cambios en calidad de datos
print("\n" + "="*80)
print("📈 MEJORAS EN CALIDAD DE DATOS:")
print("="*80)

# Comparar nulos
nulos_antes = df.isnull().sum().sum()
nulos_despues = df_final.isnull().sum().sum()
pct_nulos_antes = (nulos_antes / (df.shape[0] * df.shape[1]) * 100)
pct_nulos_despues = (nulos_despues / (df_final.shape[0] * df_final.shape[1]) * 100)

print(f"\n1. Valores Nulos:")
print(f"   Antes: {nulos_antes} ({pct_nulos_antes:.2f}%)")
print(f"   Después: {nulos_despues} ({pct_nulos_despues:.2f}%)")
print(f"   Cambio: {nulos_antes - nulos_despues} valores {'eliminados' if nulos_despues < nulos_antes else 'añadidos'}")

print(f"\n2. Duplicados:")
print(f"   Antes: {df.duplicated().sum()}")
print(f"   Después: {df_final.duplicated().sum()}")

print(f"\n3. Filas:")
print(f"   Antes: {len(df)}")
print(f"   Después: {len(df_final)}")
print(f"   Eliminadas: {len(df) - len(df_final)} ({(len(df) - len(df_final))/len(df)*100:.2f}%)")

print(f"\n4. Columnas:")
print(f"   Antes: {df.shape[1]}")
print(f"   Después: {df_final.shape[1]}")
print(f"   Eliminadas: {df.shape[1] - df_final.shape[1]}")

print("\n✅ Dataset limpio y listo para análisis exploratorio detallado")
print("="*80)


## 5. Visualización de Distribuciones de Variables Numéricas

Histogramas y gráficos de densidad para cada variable numérica.

In [ ]:
if numeric_cols:
    print(f"\n📈 Visualizando {len(numeric_cols)} variable(s) numérica(s)...\n")
    
    # Calcular número de filas y columnas para subplot
    n_cols = min(3, len(numeric_cols))
    n_rows = (len(numeric_cols) + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 4*n_rows))
    axes = axes.flatten() if len(numeric_cols) > 1 else [axes]
    
    for idx, col in enumerate(numeric_cols):
        ax = axes[idx]
        
        # Histograma con distribución normal
        ax.hist(df[col].dropna(), bins=30, alpha=0.7, edgecolor='black', color='skyblue')
        ax2 = ax.twinx()
        df[col].dropna().plot(kind='density', ax=ax2, color='red', linewidth=2)
        
        ax.set_xlabel(col)
        ax.set_ylabel('Frecuencia', color='skyblue')
        ax2.set_ylabel('Densidad', color='red')
        ax.set_title(f'Distribución de {col}')
        ax.grid(alpha=0.3)
    
    # Eliminar subplots vacíos
    for idx in range(len(numeric_cols), len(axes)):
        fig.delaxes(axes[idx])
    
    plt.tight_layout()
    plt.show()
else:
    print("No hay variables numéricas para visualizar.")

## 5.5 Análisis de Tipo de Distribución (Skewness y Kurtosis)

Interpretación estadística de la forma de las distribuciones mediante asimetría y curtosis.


In [17]:
# Análisis detallado de distribuciones con interpretación
print("\n" + "="*80)
print("ANÁLISIS DE TIPO DE DISTRIBUCIÓN")
print("="*80 + "\n")

# Usar df_clean si existe, sino df
df_dist = df_clean if 'df_clean' in locals() else df
numeric_cols_dist = df_dist.select_dtypes(include=[np.number]).columns.tolist()

if numeric_cols_dist:
    # Calcular métricas de distribución
    distribution_analysis = []
    
    for col in numeric_cols_dist:
        skew = df_dist[col].skew()
        kurt = df_dist[col].kurtosis()
        
        # Interpretar skewness
        if abs(skew) < 0.5:
            skew_interp = "Simétrica"
        elif skew < -0.5:
            skew_interp = "Asimétrica izquierda (negativa)"
        else:
            skew_interp = "Asimétrica derecha (positiva)"
        
        # Interpretar kurtosis
        if abs(kurt) < 0.5:
            kurt_interp = "Mesocúrtica (normal)"
        elif kurt < -0.5:
            kurt_interp = "Platicúrtica (aplanada)"
        else:
            kurt_interp = "Leptocúrtica (puntiaguda)"
        
        # Tipo de distribución sugerido
        if abs(skew) < 0.5 and abs(kurt) < 0.5:
            dist_type = "Normal"
        elif skew > 1:
            dist_type = "Log-normal o Exponencial"
        elif skew < -1:
            dist_type = "Uniforme o Beta"
        else:
            dist_type = "Asimétrica"
        
        distribution_analysis.append({
            'Variable': col,
            'Skewness': round(skew, 3),
            'Interpretación Skew': skew_interp,
            'Kurtosis': round(kurt, 3),
            'Interpretación Kurt': kurt_interp,
            'Tipo Sugerido': dist_type
        })
    
    # Crear DataFrame y mostrar
    dist_df = pd.DataFrame(distribution_analysis)
    
    print("📊 MÉTRICAS DE DISTRIBUCIÓN POR VARIABLE:")
    print("-"*80)
    print(dist_df.to_string(index=False))
    
    # Resumen por categoría
    print("\n\n📈 RESUMEN DE DISTRIBUCIONES:")
    print("-"*80)
    
    # Contar tipos de distribución
    normal_count = dist_df[dist_df['Tipo Sugerido'] == 'Normal'].shape[0]
    lognormal_count = dist_df[dist_df['Tipo Sugerido'] == 'Log-normal o Exponencial'].shape[0]
    asym_count = dist_df[dist_df['Tipo Sugerido'] == 'Asimétrica'].shape[0]
    other_count = len(dist_df) - normal_count - lognormal_count - asym_count
    
    print(f"\n1. Distribuciones Normales: {normal_count} variable(s)")
    if normal_count > 0:
        normal_vars = dist_df[dist_df['Tipo Sugerido'] == 'Normal']['Variable'].tolist()
        for var in normal_vars:
            print(f"   • {var}")
    
    print(f"\n2. Distribuciones Asimétricas Positivas (sesgadas derecha): {lognormal_count} variable(s)")
    if lognormal_count > 0:
        lognormal_vars = dist_df[dist_df['Tipo Sugerido'] == 'Log-normal o Exponencial']['Variable'].tolist()
        for var in lognormal_vars:
            print(f"   • {var}")
            print(f"     Skewness: {dist_df[dist_df['Variable']==var]['Skewness'].values[0]}")
    
    print(f"\n3. Distribuciones Asimétricas (otras): {asym_count} variable(s)")
    if asym_count > 0:
        asym_vars = dist_df[dist_df['Tipo Sugerido'] == 'Asimétrica']['Variable'].tolist()
        for var in asym_vars:
            print(f"   • {var}")
    
    # Recomendaciones de transformación
    print("\n\n💡 RECOMENDACIONES DE TRANSFORMACIÓN:")
    print("-"*80)
    
    transform_recommendations = []
    
    for idx, row in dist_df.iterrows():
        if row['Skewness'] > 1:
            transform_recommendations.append({
                'Variable': row['Variable'],
                'Problema': f"Asimetría positiva fuerte (skew={row['Skewness']})",
                'Transformación Sugerida': 'Logarítmica: log(x + 1) o Box-Cox',
                'Razón': 'Reducir asimetría y acercar a distribución normal'
            })
        elif row['Skewness'] < -1:
            transform_recommendations.append({
                'Variable': row['Variable'],
                'Problema': f"Asimetría negativa fuerte (skew={row['Skewness']})",
                'Transformación Sugerida': 'Cuadrado: x^2 o reflexión + log',
                'Razón': 'Reducir asimetría negativa'
            })
        
        if row['Kurtosis'] > 3:
            transform_recommendations.append({
                'Variable': row['Variable'],
                'Problema': f"Curtosis excesiva (kurt={row['Kurtosis']})",
                'Transformación Sugerida': 'Recorte de outliers o Winsorización',
                'Razón': 'Reducir influencia de valores extremos'
            })
    
    if transform_recommendations:
        print(f"\n{len(transform_recommendations)} transformación(es) recomendada(s):\n")
        
        for idx, rec in enumerate(transform_recommendations, 1):
            print(f"{idx}. {rec['Variable']}")
            print(f"   Problema: {rec['Problema']}")
            print(f"   Transformación: {rec['Transformación Sugerida']}")
            print(f"   Razón: {rec['Razón']}")
            print()
    else:
        print("\n✓ No se requieren transformaciones mayores")
        print("  Las distribuciones son razonablemente normales o simétricas")
    
    # Interpretación para modelos
    print("\n📝 IMPLICACIONES PARA MODELADO:")
    print("-"*80)
    print("""
    1. Variables con distribución normal: Ideales para modelos lineales y regresión
    
    2. Variables asimétricas: Considerar transformaciones antes de modelos lineales
       • Skewness > 1: Aplicar log transform
       • Skewness < -1: Aplicar transformación potencia
    
    3. Variables con alta curtosis: Pueden contener outliers influyentes
       • Verificar outliers antes de modelar
       • Considerar modelos robustos (e.g., Random Forest, Gradient Boosting)
    
    4. Modelos basados en árboles (Random Forest, XGBoost): No requieren
       transformaciones, son invariantes a distribuciones
    """)
    
else:
    print("\n⚠️ No hay variables numéricas para analizar distribuciones")

print("\n" + "="*80)



ANÁLISIS DE TIPO DE DISTRIBUCIÓN

📊 MÉTRICAS DE DISTRIBUCIÓN POR VARIABLE:
--------------------------------------------------------------------------------
            Variable  Skewness Interpretación Skew  Kurtosis     Interpretación Kurt Tipo Sugerido
                 Age     0.046           Simétrica    -1.189 Platicúrtica (aplanada)    Asimétrica
                 BMI    -0.027           Simétrica    -1.185 Platicúrtica (aplanada)    Asimétrica
  AlcoholConsumption     0.018           Simétrica    -1.203 Platicúrtica (aplanada)    Asimétrica
    PhysicalActivity     0.045           Simétrica    -1.179 Platicúrtica (aplanada)    Asimétrica
         DietQuality    -0.012           Simétrica    -1.229 Platicúrtica (aplanada)    Asimétrica
        SleepQuality    -0.070           Simétrica    -1.212 Platicúrtica (aplanada)    Asimétrica
          SystolicBP     0.010           Simétrica    -1.198 Platicúrtica (aplanada)    Asimétrica
         DiastolicBP    -0.054           Simétrica 

## 6. Análisis de Outliers

Detección de outliers usando el método del Rango Intercuartílico (IQR).

In [ ]:
# Detección de outliers usando IQR
print("\n" + "="*80)
print("ANÁLISIS DE OUTLIERS (Método IQR)")
print("="*80 + "\n")

outlier_summary = []

if numeric_cols:
    for col in numeric_cols:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
        n_outliers = len(outliers)
        
        if n_outliers > 0:
            pct_outliers = round((n_outliers / len(df) * 100), 2)
            outlier_summary.append({
                'Columna': col,
                'Outliers': n_outliers,
                '% Outliers': pct_outliers,
                'Rango Válido': f"[{lower_bound:.2f}, {upper_bound:.2f}]"
            })
            print(f"⚠️  {col}: {n_outliers} outliers ({pct_outliers}%)")
    
    if outlier_summary:
        outlier_df = pd.DataFrame(outlier_summary)
        print("\n" + outlier_df.to_string(index=False))
        
        # Boxplot de variables con outliers - Disposición vertical
        if len(outlier_summary) > 0:
            n_plots = len(outlier_summary)
            n_cols = 2  # 2 columnas
            n_rows = (n_plots + n_cols - 1) // n_cols  # Calcula filas necesarias
            
            fig, axes = plt.subplots(n_rows, n_cols, figsize=(12, 4*n_rows))
            axes = axes.flatten()  # Convierte a array 1D para indexar fácilmente
            
            for idx, item in enumerate(outlier_summary):
                col = item['Columna']
                axes[idx].boxplot(df[col].dropna(), vert=True)
                axes[idx].set_ylabel(col)
                axes[idx].set_title(f'Boxplot de {col}')
                axes[idx].grid(alpha=0.3, axis='y')
            
            # Eliminar subplots vacíos si los hay
            for idx in range(len(outlier_summary), len(axes)):
                fig.delaxes(axes[idx])
            
            plt.tight_layout()
            plt.show()
    else:
        print("\n✓ No se detectaron outliers significativos en el dataset")
else:
    print("No hay variables numéricas para analizar outliers.")

## 7. Análisis de Variables Categóricas

Gráficos de barras para variables categóricas.

In [ ]:
if categorical_cols:
    print(f"\n📊 Visualizando {len(categorical_cols)} variable(s) categórica(s)...\n")
    
    # Calcular número de filas y columnas para subplot
    n_cols = min(3, len(categorical_cols))
    n_rows = (len(categorical_cols) + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 4*n_rows))
    axes = axes.flatten() if len(categorical_cols) > 1 else [axes]
    
    for idx, col in enumerate(categorical_cols):
        ax = axes[idx]
        
        # Contar valores y crear gráfico
        value_counts = df[col].value_counts()
        
        # Limitar a 10 categorías para mejor visualización
        if len(value_counts) > 10:
            value_counts = value_counts.head(10)
            title = f'Top 10 - {col} (hay más categorías)'
        else:
            title = f'Distribución de {col}'
        
        value_counts.plot(kind='bar', ax=ax, color='teal', edgecolor='black')
        ax.set_title(title)
        ax.set_xlabel(col)
        ax.set_ylabel('Frecuencia')
        ax.grid(axis='y', alpha=0.3)
        plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, ha='right')
    
    # Eliminar subplots vacíos
    for idx in range(len(categorical_cols), len(axes)):
        fig.delaxes(axes[idx])
    
    plt.tight_layout()
    plt.show()
else:
    print("No hay variables categóricas para visualizar.")

## 7.5 Análisis Profundo de Variables Categóricas (Countplots y Tablas Pivote)

Exploración adicional con countplots de seaborn y tablas de contingencia.


In [ ]:
# Análisis profundo de variables categóricas con countplot y tablas pivote
print("\n" + "="*80)
print("ANÁLISIS PROFUNDO DE VARIABLES CATEGÓRICAS")
print("="*80 + "\n")

# Usar df_clean si existe
df_cat = df_clean if 'df_clean' in locals() else df
categorical_cols_cat = df_cat.select_dtypes(include=['object', 'category']).columns.tolist()

# Identificar target
target_for_analysis = target_column if 'target_column' in locals() and target_column else None

if categorical_cols_cat:
    # 1. COUNTPLOTS CON SEABORN
    print("1️⃣ COUNTPLOTS DE VARIABLES CATEGÓRICAS:")
    print("-"*80)
    
    # Seleccionar primeras 6 variables para countplot
    cat_for_plot = categorical_cols_cat[:min(6, len(categorical_cols_cat))]
    
    if target_for_analysis and target_for_analysis in df_cat.columns:
        # Countplots con hue (color por target)
        n_cols = 2
        n_rows = (len(cat_for_plot) + n_cols - 1) // n_cols
        
        fig, axes = plt.subplots(n_rows, n_cols, figsize=(14, 4*n_rows))
        axes = axes.flatten() if len(cat_for_plot) > 1 else [axes]
        
        for idx, col in enumerate(cat_for_plot):
            if col != target_for_analysis:  # No graficar target contra sí mismo
                ax = axes[idx]
                
                # Countplot con hue
                sns.countplot(data=df_cat, x=col, hue=target_for_analysis, ax=ax, palette='Set2')
                ax.set_title(f'Distribución de {col} por {target_for_analysis}')
                ax.set_xlabel(col)
                ax.set_ylabel('Frecuencia')
                ax.legend(title=target_for_analysis)
                plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, ha='right')
        
        # Eliminar subplots vacíos
        for idx in range(len(cat_for_plot), len(axes)):
            fig.delaxes(axes[idx])
        
        plt.tight_layout()
        plt.show()
    else:
        # Countplots simples sin hue
        n_cols = 3
        n_rows = (len(cat_for_plot) + n_cols - 1) // n_cols
        
        fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 4*n_rows))
        axes = axes.flatten() if len(cat_for_plot) > 1 else [axes]
        
        for idx, col in enumerate(cat_for_plot):
            ax = axes[idx]
            
            sns.countplot(data=df_cat, x=col, ax=ax, palette='viridis')
            ax.set_title(f'Distribución de {col}')
            ax.set_xlabel(col)
            ax.set_ylabel('Frecuencia')
            plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, ha='right')
        
        # Eliminar subplots vacíos
        for idx in range(len(cat_for_plot), len(axes)):
            fig.delaxes(axes[idx])
        
        plt.tight_layout()
        plt.show()
    
    print(f"✅ Countplots generados para {len(cat_for_plot)} variable(s)\n")
    
    # 2. TABLAS PIVOTE (CROSSTAB)
    if target_for_analysis and target_for_analysis in df_cat.columns:
        print("\n2️⃣ TABLAS PIVOTE (CROSSTAB) - Variables vs Target:")
        print("-"*80)
        
        # Seleccionar variables categóricas (excluyendo target)
        cat_for_pivot = [col for col in categorical_cols_cat if col != target_for_analysis][:min(5, len(categorical_cols_cat)-1)]
        
        for col in cat_for_pivot:
            print(f"\n📊 Tabla de Contingencia: {col} vs {target_for_analysis}")
            print("-"*40)
            
            # Crear crosstab con totales
            crosstab = pd.crosstab(
                df_cat[col], 
                df_cat[target_for_analysis], 
                margins=True,
                margins_name='Total'
            )
            print(crosstab)
            
            # Tabla de contingencia con porcentajes (por fila)
            print(f"\n📊 Porcentajes por Fila ({col}):")
            print("-"*40)
            crosstab_pct = pd.crosstab(
                df_cat[col], 
                df_cat[target_for_analysis], 
                normalize='index'
            ) * 100
            print(crosstab_pct.round(2))
            
            # Análisis de asociación (Chi-cuadrado)
            from scipy import stats
            
            contingency_table = pd.crosstab(df_cat[col], df_cat[target_for_analysis])
            chi2, p_value, dof, expected = stats.chi2_contingency(contingency_table)
            
            print(f"\n🔬 Test de Independencia (Chi-cuadrado):")
            print(f"   Chi²: {chi2:.4f}")
            print(f"   p-value: {p_value:.4f}")
            print(f"   Grados de libertad: {dof}")
            
            if p_value < 0.001:
                print(f"   ✅ Asociación ALTAMENTE SIGNIFICATIVA (p < 0.001) ***")
            elif p_value < 0.01:
                print(f"   ✅ Asociación MUY SIGNIFICATIVA (p < 0.01) **")
            elif p_value < 0.05:
                print(f"   ✅ Asociación SIGNIFICATIVA (p < 0.05) *")
            else:
                print(f"   ❌ NO hay asociación significativa (p >= 0.05)")
            
            print("\n" + "="*80)
    
    # 3. VALUE_COUNTS DETALLADO
    print("\n3️⃣ VALUE_COUNTS DETALLADO:")
    print("-"*80)
    
    for col in categorical_cols_cat[:min(5, len(categorical_cols_cat))]:
        print(f"\n📊 {col}")
        print("-"*40)
        
        value_counts = df_cat[col].value_counts()
        value_pct = df_cat[col].value_counts(normalize=True) * 100
        
        # Crear DataFrame combinado
        value_summary = pd.DataFrame({
            'Frecuencia': value_counts,
            'Porcentaje (%)': value_pct.round(2)
        })
        
        print(value_summary)
        
        # Estadísticas adicionales
        print(f"\n   Estadísticas:")
        print(f"   • Total categorías: {df_cat[col].nunique()}")
        print(f"   • Moda: {df_cat[col].mode()[0] if len(df_cat[col].mode()) > 0 else 'N/A'}")
        print(f"   • Categoría más frecuente: {value_counts.idxmax()} ({value_counts.max()} veces, {value_pct.max():.2f}%)")
        print(f"   • Categoría menos frecuente: {value_counts.idxmin()} ({value_counts.min()} veces, {value_pct.min():.2f}%)")
        
        # Calcular entropía (medida de diversidad)
        entropy = -sum(value_pct/100 * np.log2(value_pct/100))
        max_entropy = np.log2(df_cat[col].nunique())
        normalized_entropy = entropy / max_entropy if max_entropy > 0 else 0
        
        print(f"   • Entropía normalizada: {normalized_entropy:.3f} (0=uniforme, 1=máxima diversidad)")
        
        if normalized_entropy < 0.3:
            print(f"   ⚠️ Variable poco diversa (dominada por pocas categorías)")
        elif normalized_entropy > 0.8:
            print(f"   ✓ Variable bien diversificada")
    
    print("\n" + "="*80)
    print("✅ Análisis de variables categóricas completado")
    print("="*80)
    
else:
    print("\n⚠️ No hay variables categóricas para analizar")



## 8. Análisis de Correlación

Correlación entre variables numéricas y heatmap de correlación.

In [ ]:
if len(numeric_cols) > 1:
    print("\n" + "="*80)
    print("MATRIZ DE CORRELACIÓN")
    print("="*80 + "\n")
    
    correlation_matrix = df[numeric_cols].corr()
    print(correlation_matrix.round(3))
    
    # Heatmap de correlación - Mejorado para legibilidad
    fig_size = max(14, len(numeric_cols) + 2)
    fig, ax = plt.subplots(figsize=(fig_size, fig_size))
    
    sns.heatmap(correlation_matrix, 
                annot=True, 
                fmt='.2f', 
                cmap='coolwarm', 
                center=0, 
                square=True, 
                ax=ax, 
                cbar_kws={"shrink": 0.8},
                vmin=-1, 
                vmax=1,
                annot_kws={"size": 8},
                linewidths=0.5,
                linecolor='gray')
    
    ax.set_title('Matriz de Correlación de Variables Numéricas', fontsize=14, pad=20)
    
    # Rotar etiquetas para mejor legibilidad
    plt.xticks(rotation=45, ha='right', fontsize=9)
    plt.yticks(rotation=0, fontsize=9)
    
    plt.tight_layout()
    plt.show()
    
    # Encontrar correlaciones fuertes (>0.7 o <-0.7)
    print("\n" + "-"*80)
    print("CORRELACIONES FUERTES (|r| > 0.7):")
    print("-"*80 + "\n")
    
    strong_corr = []
    for i in range(len(correlation_matrix.columns)):
        for j in range(i+1, len(correlation_matrix.columns)):
            if abs(correlation_matrix.iloc[i, j]) > 0.7:
                strong_corr.append({
                    'Variable 1': correlation_matrix.columns[i],
                    'Variable 2': correlation_matrix.columns[j],
                    'Correlación': correlation_matrix.iloc[i, j].round(3)
                })
    
    if strong_corr:
        strong_corr_df = pd.DataFrame(strong_corr).sort_values('Correlación', key=abs, ascending=False)
        print(strong_corr_df.to_string(index=False))
    else:
        print("No hay correlaciones fuertes entre variables.")
else:
    print("\nNo hay suficientes variables numéricas para calcular correlación.")

## 8.5 Análisis Bivariado con Variable Objetivo

Exploración de la relación entre cada feature y la variable objetivo (Diagnosis).

In [ ]:
# Verificar si existe variable objetivo (Diagnosis o similar)
target_col = None
possible_targets = ['Diagnosis', 'diagnosis', 'target', 'Target', 'label', 'Label']

for col in possible_targets:
    if col in df.columns:
        target_col = col
        break

if target_col and target_col in df.columns:
    print("\n" + "="*80)
    print(f"ANÁLISIS BIVARIADO: FEATURES vs {target_col}")
    print("="*80)
    
    # Distribución de la variable objetivo
    print(f"\n📊 Distribución de {target_col}:")
    target_dist = df[target_col].value_counts()
    print(target_dist)
    print(f"\nProporción:")
    for label, count in target_dist.items():
        print(f"   Clase {label}: {count/len(df)*100:.1f}%")
    
    # Análisis de variables numéricas vs target
    if numeric_cols:
        print(f"\n📈 Análisis de Variables Numéricas por {target_col}:")
        print("-"*80)
        
        # Seleccionar hasta 6 variables más importantes (por correlación si existe)
        features_to_analyze = numeric_cols[:min(6, len(numeric_cols))]
        
        n_cols = 3
        n_rows = (len(features_to_analyze) + n_cols - 1) // n_cols
        
        fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 4*n_rows))
        axes = axes.flatten() if len(features_to_analyze) > 1 else [axes]
        
        for idx, col in enumerate(features_to_analyze):
            ax = axes[idx]
            
            # Boxplot por clase
            df_plot = df[[col, target_col]].dropna()
            classes = sorted(df_plot[target_col].unique())
            
            data_by_class = [df_plot[df_plot[target_col] == c][col].values for c in classes]
            
            bp = ax.boxplot(data_by_class, labels=classes, patch_artist=True)
            
            # Colorear boxplots
            colors = ['lightblue', 'lightcoral', 'lightgreen', 'lightyellow']
            for patch, color in zip(bp['boxes'], colors):
                patch.set_facecolor(color)
            
            ax.set_xlabel(target_col)
            ax.set_ylabel(col)
            ax.set_title(f'{col} por {target_col}')
            ax.grid(alpha=0.3, axis='y')
        
        # Eliminar subplots vacíos
        for idx in range(len(features_to_analyze), len(axes)):
            fig.delaxes(axes[idx])
        
        plt.tight_layout()
        plt.show()
        
        # Tests estadísticos de significancia (ejemplo con primeras 3 variables)
        print(f"\n🔬 Tests Estadísticos (t-test para diferencias entre grupos):")
        print("-"*80)
        
        from scipy import stats
        
        for col in features_to_analyze[:3]:
            df_test = df[[col, target_col]].dropna()
            classes = sorted(df_test[target_col].unique())
            
            if len(classes) == 2:
                # t-test para 2 grupos
                group1 = df_test[df_test[target_col] == classes[0]][col]
                group2 = df_test[df_test[target_col] == classes[1]][col]
                
                t_stat, p_value = stats.ttest_ind(group1, group2)
                
                significance = "***" if p_value < 0.001 else "**" if p_value < 0.01 else "*" if p_value < 0.05 else "ns"
                
                print(f"\n{col}:")
                print(f"   Media Clase {classes[0]}: {group1.mean():.3f}")
                print(f"   Media Clase {classes[1]}: {group2.mean():.3f}")
                print(f"   t-statistic: {t_stat:.3f}")
                print(f"   p-value: {p_value:.4f} {significance}")
                
                if p_value < 0.05:
                    print(f"   ✓ Diferencia significativa entre grupos")
                else:
                    print(f"   ✗ No hay diferencia significativa")
    
    # Análisis de variables categóricas vs target
    if categorical_cols and target_col not in categorical_cols:
        print(f"\n\n📊 Análisis de Variables Categóricas por {target_col}:")
        print("-"*80)
        
        # Seleccionar primeras 3-4 variables categóricas
        cat_to_analyze = categorical_cols[:min(4, len(categorical_cols))]
        
        for col in cat_to_analyze:
            print(f"\n{col}:")
            
            # Tabla de contingencia
            contingency = pd.crosstab(df[col], df[target_col], margins=True)
            print(contingency)
            
            # Chi-cuadrado test
            if len(df[col].unique()) > 1:
                contingency_no_margins = pd.crosstab(df[col], df[target_col])
                chi2, p_value, dof, expected = stats.chi2_contingency(contingency_no_margins)
                
                significance = "***" if p_value < 0.001 else "**" if p_value < 0.01 else "*" if p_value < 0.05 else "ns"
                
                print(f"\nChi-cuadrado: {chi2:.3f}, p-value: {p_value:.4f} {significance}")
                
                if p_value < 0.05:
                    print(f"✓ Asociación significativa con {target_col}")
                else:
                    print(f"✗ No hay asociación significativa")
                print("-"*40)
else:
    print("\n⚠️ No se encontró variable objetivo (Diagnosis) en el dataset")
    print("   Si existe con otro nombre, ajustar la variable 'target_col'")


## 8.6 Análisis Multivariado: Pairplot

Exploración de relaciones entre múltiples variables simultáneamente.

In [ ]:
if target_col and len(numeric_cols) > 1:
    print("\n" + "="*80)
    print("ANÁLISIS MULTIVARIADO: PAIRPLOT")
    print("="*80)
    
    # Seleccionar top 5-6 variables con mayor correlación con el target (si es numérico)
    # O simplemente las primeras 5-6 variables numéricas
    
    max_features = 5  # Límite para que el pairplot sea legible
    
    if target_col in numeric_cols:
        # Si target es numérico, seleccionar por correlación
        correlations = df[numeric_cols].corr()[target_col].abs().sort_values(ascending=False)
        top_features = correlations[1:max_features+1].index.tolist()  # Excluir el target mismo
        features_for_pairplot = top_features + [target_col]
    else:
        # Si target es categórico, seleccionar primeras N features
        features_for_pairplot = numeric_cols[:min(max_features, len(numeric_cols))]
    
    print(f"\n📊 Creando pairplot con {len(features_for_pairplot)} variables:")
    for feat in features_for_pairplot:
        print(f"   • {feat}")
    
    # Crear subset del dataframe
    df_pairplot = df[features_for_pairplot + ([target_col] if target_col not in features_for_pairplot else [])].dropna()
    
    print(f"\n⏳ Generando pairplot (esto puede tomar unos segundos)...")
    
    # Crear pairplot
    if target_col in df.columns and df[target_col].dtype in ['object', 'int64'] and df[target_col].nunique() <= 5:
        # Si target es categórico con pocas clases, usar como hue
        pairplot = sns.pairplot(
            df_pairplot, 
            hue=target_col,
            diag_kind='kde',
            plot_kws={'alpha': 0.6, 's': 30},
            height=2.5
        )
        pairplot.fig.suptitle(f'Pairplot de Variables Numéricas por {target_col}', y=1.02, fontsize=14)
    else:
        # Si no, pairplot simple
        pairplot = sns.pairplot(
            df_pairplot,
            diag_kind='kde',
            plot_kws={'alpha': 0.6},
            height=2.5
        )
        pairplot.fig.suptitle('Pairplot de Variables Numéricas Principales', y=1.02, fontsize=14)
    
    plt.tight_layout()
    plt.show()
    
    print("\n✅ Pairplot completado")
    print("\n💡 Interpretación:")
    print("   • Diagonal: Distribución de cada variable")
    print("   • Fuera de diagonal: Scatter plots entre pares de variables")
    if target_col in df.columns and df[target_col].dtype in ['object', 'int64'] and df[target_col].nunique() <= 5:
        print(f"   • Colores: Representan diferentes clases de {target_col}")
        print("   • Buscar: Separación clara entre colores indica poder predictivo")
    
elif len(numeric_cols) > 1:
    print("\n⚠️ Variable objetivo no encontrada. Creando pairplot sin clasificación por color.")
    
    features_for_pairplot = numeric_cols[:min(5, len(numeric_cols))]
    df_pairplot = df[features_for_pairplot].dropna()
    
    pairplot = sns.pairplot(df_pairplot, diag_kind='kde', height=2.5)
    pairplot.fig.suptitle('Pairplot de Variables Numéricas', y=1.02)
    plt.tight_layout()
    plt.show()
else:
    print("\n⚠️ No hay suficientes variables numéricas para crear un pairplot")


## 8.7 Sugerencias de Features Derivados

Basado en el análisis exploratorio, se identifican oportunidades para crear características calculadas.

In [ ]:
print("\n" + "="*80)
print("FEATURES DERIVADOS POTENCIALES")
print("="*80)

derived_features = []

# Verificar columnas específicas del dataset de Alzheimer
health_indicators = []
lifestyle_indicators = []
cognitive_indicators = []
cardiovascular_indicators = []

# Clasificar columnas por categoría
for col in df.columns:
    col_lower = col.lower()
    
    if any(x in col_lower for x in ['cholesterol', 'bp', 'blood', 'systolic', 'diastolic']):
        cardiovascular_indicators.append(col)
    elif any(x in col_lower for x in ['mmse', 'memory', 'cognitive', 'functional', 'adl', 'confusion']):
        cognitive_indicators.append(col)
    elif any(x in col_lower for x in ['smoking', 'alcohol', 'physical', 'diet', 'sleep', 'bmi']):
        lifestyle_indicators.append(col)
    elif any(x in col_lower for x in ['diabetes', 'hypertension', 'cardiovascular', 'depression']):
        health_indicators.append(col)

print("\n📋 CATEGORÍAS DE VARIABLES IDENTIFICADAS:")
print(f"   • Indicadores de Salud: {len(health_indicators)}")
print(f"   • Indicadores de Estilo de Vida: {len(lifestyle_indicators)}")
print(f"   • Indicadores Cognitivos: {len(cognitive_indicators)}")
print(f"   • Indicadores Cardiovasculares: {len(cardiovascular_indicators)}")

print("\n\n💡 FEATURES DERIVADOS SUGERIDOS:")
print("="*80)

# 1. Ratios de colesterol
if 'CholesterolLDL' in df.columns and 'CholesterolHDL' in df.columns:
    derived_features.append({
        'nombre': 'Cholesterol_Ratio_LDL_HDL',
        'fórmula': 'CholesterolLDL / CholesterolHDL',
        'justificación': 'Indicador de riesgo cardiovascular. Ratio alto asociado con mayor riesgo.',
        'implementación': 'df["Cholesterol_Ratio"] = df["CholesterolLDL"] / df["CholesterolHDL"]'
    })

if 'CholesterolTotal' in df.columns and 'CholesterolHDL' in df.columns:
    derived_features.append({
        'nombre': 'Cholesterol_Total_HDL_Ratio',
        'fórmula': 'CholesterolTotal / CholesterolHDL',
        'justificación': 'Otro indicador cardiovascular. Valores normales < 5.',
        'implementación': 'df["Total_HDL_Ratio"] = df["CholesterolTotal"] / df["CholesterolHDL"]'
    })

# 2. Presión arterial media
if 'SystolicBP' in df.columns and 'DiastolicBP' in df.columns:
    derived_features.append({
        'nombre': 'Mean_Arterial_Pressure',
        'fórmula': 'DiastolicBP + (SystolicBP - DiastolicBP) / 3',
        'justificación': 'Presión arterial media, mejor indicador de perfusión cerebral.',
        'implementación': 'df["MAP"] = df["DiastolicBP"] + (df["SystolicBP"] - df["DiastolicBP"]) / 3'
    })

# 3. IMC categorizado
if 'BMI' in df.columns:
    derived_features.append({
        'nombre': 'BMI_Category',
        'fórmula': 'Categorización: Bajo(<18.5), Normal(18.5-24.9), Sobrepeso(25-29.9), Obeso(>=30)',
        'justificación': 'Categorías clínicas de IMC más interpretables que valor continuo.',
        'implementación': 'pd.cut(df["BMI"], bins=[0, 18.5, 25, 30, 100], labels=["Bajo", "Normal", "Sobrepeso", "Obeso"])'
    })

# 4. Score de riesgo cardiovascular
if health_indicators:
    derived_features.append({
        'nombre': 'Cardiovascular_Risk_Score',
        'fórmula': f'Suma de: {", ".join(health_indicators[:5])}',
        'justificación': 'Agregación de factores de riesgo cardiovascular.',
        'implementación': f'df[{health_indicators[:5]}].sum(axis=1)'
    })

# 5. Score de estilo de vida saludable
if lifestyle_indicators:
    derived_features.append({
        'nombre': 'Healthy_Lifestyle_Score',
        'fórmula': 'Combinación ponderada de hábitos saludables',
        'justificación': 'Agregación de factores de estilo de vida que afectan salud cerebral.',
        'implementación': 'Normalizar y sumar: PhysicalActivity, DietQuality, SleepQuality (invertir Smoking y AlcoholConsumption)'
    })

# 6. Score cognitivo compuesto
if cognitive_indicators:
    derived_features.append({
        'nombre': 'Cognitive_Impairment_Score',
        'fórmula': f'Combinación de: {", ".join(cognitive_indicators[:4])}',
        'justificación': 'Indicador agregado de deterioro cognitivo.',
        'implementación': 'Suma o promedio de indicadores cognitivos'
    })

# 7. Interacciones de edad
if 'Age' in df.columns:
    derived_features.append({
        'nombre': 'Age_Squared',
        'fórmula': 'Age ** 2',
        'justificación': 'Capturar relación no lineal entre edad y riesgo.',
        'implementación': 'df["Age_Squared"] = df["Age"] ** 2'
    })
    
    if 'FamilyHistoryAlzheimers' in df.columns:
        derived_features.append({
            'nombre': 'Age_Family_History_Interaction',
            'fórmula': 'Age * FamilyHistoryAlzheimers',
            'justificación': 'Interacción entre edad y predisposición genética.',
            'implementación': 'df["Age_FH_Interaction"] = df["Age"] * df["FamilyHistoryAlzheimers"]'
        })

# 8. Binning de variables continuas
if 'Age' in df.columns:
    derived_features.append({
        'nombre': 'Age_Group',
        'fórmula': 'Categorías: <65, 65-74, 75-84, 85+',
        'justificación': 'Grupos de edad clínicamente relevantes para Alzheimer.',
        'implementación': 'pd.cut(df["Age"], bins=[0, 65, 75, 85, 120], labels=["<65", "65-74", "75-84", "85+"])'
    })

# Mostrar tabla de features sugeridos
if derived_features:
    print(f"\nTotal de features derivados sugeridos: {len(derived_features)}\n")
    
    for idx, feature in enumerate(derived_features, 1):
        print(f"{idx}. {feature['nombre']}")
        print(f"   Fórmula: {feature['fórmula']}")
        print(f"   Justificación: {feature['justificación']}")
        print(f"   Implementación: {feature['implementación']}")
        print()
    
    print("\n" + "="*80)
    print("RECOMENDACIONES DE IMPLEMENTACIÓN:")
    print("="*80)
    print("""
    1. Implementar estos features en el script 'ft_engineering.py'
    2. Crear una función 'create_derived_features()' antes del preprocesamiento
    3. Validar que los features derivados no tengan valores infinitos o NaN
    4. Evaluar la importancia de estos features después del entrenamiento
    5. Considerar regularización si se agregan muchos features (evitar overfitting)
    """)
    
    # Ejemplo de implementación
    print("\n📝 EJEMPLO DE CÓDIGO PARA IMPLEMENTACIÓN:")
    print("-"*80)
    print("""
def create_derived_features(df):
    '''Crea features derivados basados en análisis EDA'''
    df_new = df.copy()
    
    # Ratio LDL/HDL
    if 'CholesterolLDL' in df.columns and 'CholesterolHDL' in df.columns:
        df_new['Cholesterol_Ratio'] = df_new['CholesterolLDL'] / df_new['CholesterolHDL']
    
    # Presión arterial media
    if 'SystolicBP' in df.columns and 'DiastolicBP' in df.columns:
        df_new['MAP'] = df_new['DiastolicBP'] + (df_new['SystolicBP'] - df_new['DiastolicBP']) / 3
    
    # IMC categorizado
    if 'BMI' in df.columns:
        df_new['BMI_Category'] = pd.cut(df_new['BMI'], 
                                         bins=[0, 18.5, 25, 30, 100], 
                                         labels=['Bajo', 'Normal', 'Sobrepeso', 'Obeso'])
    
    # Edad al cuadrado
    if 'Age' in df.columns:
        df_new['Age_Squared'] = df_new['Age'] ** 2
    
    return df_new
    """)
else:
    print("\n⚠️ No se identificaron oportunidades claras para features derivados.")
    print("   Esto puede deberse a que el dataset no contiene las columnas esperadas.")

print("\n" + "="*80)


## 8.8 Reglas de Validación de Datos

Identificación y documentación de reglas de negocio y restricciones de integridad para el dataset.


In [ ]:
# Identificar y documentar reglas de validación de datos
print("\n" + "="*80)
print("REGLAS DE VALIDACIÓN DE DATOS")
print("="*80 + "\n")

# Usar df_clean si existe
df_validate = df_clean if 'df_clean' in locals() else df

# Estructura para almacenar reglas
validation_rules_dict = {}

print("📋 REGLAS DE VALIDACIÓN IDENTIFICADAS:")
print("="*80)

# REGLA 1: RANGOS VÁLIDOS PARA VARIABLES NUMÉRICAS
print("\n1️⃣ RANGOS VÁLIDOS PARA VARIABLES NUMÉRICAS:")
print("-"*80)

numeric_ranges = {
    'Age': {
        'min': 0,
        'max': 120,
        'tipo': 'Edad en años',
        'justificación': 'Rango biológico humano'
    },
    'BMI': {
        'min': 10,
        'max': 60,
        'tipo': 'Índice de Masa Corporal',
        'justificación': 'Rango clínico válido'
    },
    'SystolicBP': {
        'min': 70,
        'max': 250,
        'tipo': 'Presión arterial sistólica (mmHg)',
        'justificación': 'Rango fisiológico compatible con vida'
    },
    'DiastolicBP': {
        'min': 40,
        'max': 150,
        'tipo': 'Presión arterial diastólica (mmHg)',
        'justificación': 'Rango fisiológico compatible con vida'
    },
    'CholesterolTotal': {
        'min': 100,
        'max': 400,
        'tipo': 'Colesterol total (mg/dL)',
        'justificación': 'Rango clínico observado'
    },
    'CholesterolLDL': {
        'min': 50,
        'max': 300,
        'tipo': 'Colesterol LDL (mg/dL)',
        'justificación': 'Rango clínico observado'
    },
    'CholesterolHDL': {
        'min': 20,
        'max': 100,
        'tipo': 'Colesterol HDL (mg/dL)',
        'justificación': 'Rango clínico observado'
    },
    'CholesterolTriglycerides': {
        'min': 50,
        'max': 500,
        'tipo': 'Triglicéridos (mg/dL)',
        'justificación': 'Rango clínico observado'
    },
    'MMSE': {
        'min': 0,
        'max': 30,
        'tipo': 'Mini-Mental State Examination',
        'justificación': 'Escala de 0-30 puntos'
    },
    'FunctionalAssessment': {
        'min': 0,
        'max': 10,
        'tipo': 'Evaluación funcional',
        'justificación': 'Escala de 0-10 puntos'
    },
    'ADL': {
        'min': 0,
        'max': 10,
        'tipo': 'Activities of Daily Living',
        'justificación': 'Escala de 0-10 puntos'
    }
}

for col, rules in numeric_ranges.items():
    if col in df_validate.columns:
        print(f"\n   • {col}: [{rules['min']}, {rules['max']}]")
        print(f"     Tipo: {rules['tipo']}")
        print(f"     Justificación: {rules['justificación']}")
        
        # Validar
        out_of_range = df_validate[(df_validate[col] < rules['min']) | (df_validate[col] > rules['max'])][col].count()
        
        if out_of_range > 0:
            print(f"     ⚠️ {out_of_range} valor(es) fuera de rango")
        else:
            print(f"     ✓ Todos los valores en rango válido")
        
        validation_rules_dict[col] = rules

# REGLA 2: RELACIONES LÓGICAS ENTRE VARIABLES
print("\n\n2️⃣ RELACIONES LÓGICAS ENTRE VARIABLES:")
print("-"*80)

logical_rules = []

# Regla: Systolic BP > Diastolic BP
if 'SystolicBP' in df_validate.columns and 'DiastolicBP' in df_validate.columns:
    rule = {
        'nombre': 'Presión Sistólica > Diastólica',
        'condición': 'SystolicBP > DiastolicBP',
        'justificación': 'Principio fisiológico básico'
    }
    logical_rules.append(rule)
    
    violations = (df_validate['SystolicBP'] <= df_validate['DiastolicBP']).sum()
    print(f"\n   • {rule['nombre']}")
    print(f"     Condición: {rule['condición']}")
    print(f"     Justificación: {rule['justificación']}")
    print(f"     Violaciones: {violations} registro(s)")

# Regla: LDL + HDL ≤ Colesterol Total
if all(col in df_validate.columns for col in ['CholesterolTotal', 'CholesterolLDL', 'CholesterolHDL']):
    rule = {
        'nombre': 'LDL + HDL ≤ Colesterol Total',
        'condición': 'CholesterolLDL + CholesterolHDL ≤ CholesterolTotal * 1.1',
        'justificación': 'El total debe ser suma de componentes (con margen 10%)'
    }
    logical_rules.append(rule)
    
    violations = ((df_validate['CholesterolLDL'] + df_validate['CholesterolHDL']) > 
                  df_validate['CholesterolTotal'] * 1.1).sum()
    print(f"\n   • {rule['nombre']}")
    print(f"     Condición: {rule['condición']}")
    print(f"     Justificación: {rule['justificación']}")
    print(f"     Violaciones: {violations} registro(s)")

# Regla: BMI coherente con altura/peso (si existen)
if all(col in df_validate.columns for col in ['Height', 'Weight', 'BMI']):
    rule = {
        'nombre': 'BMI = Weight / (Height²)',
        'condición': 'BMI ≈ Weight / ((Height/100)²)',
        'justificación': 'Fórmula matemática del IMC'
    }
    logical_rules.append(rule)
    
    calculated_bmi = df_validate['Weight'] / ((df_validate['Height']/100) ** 2)
    violations = (abs(df_validate['BMI'] - calculated_bmi) > 2).sum()  # Margen de 2 unidades
    print(f"\n   • {rule['nombre']}")
    print(f"     Condición: {rule['condición']}")
    print(f"     Justificación: {rule['justificación']}")
    print(f"     Violaciones: {violations} registro(s) (diferencia > 2)")

# REGLA 3: VALORES OBLIGATORIOS (NO NULOS)
print("\n\n3️⃣ CAMPOS OBLIGATORIOS (NO NULOS):")
print("-"*80)

mandatory_fields = ['Age', 'Gender', 'Diagnosis']  # Campos que SIEMPRE deben tener valor

for field in mandatory_fields:
    if field in df_validate.columns:
        null_count = df_validate[field].isnull().sum()
        print(f"\n   • {field}")
        print(f"     Valores nulos: {null_count}")
        
        if null_count > 0:
            print(f"     ⚠️ VIOLACIÓN: Campo obligatorio con valores nulos")
        else:
            print(f"     ✓ Cumple: Sin valores nulos")

# REGLA 4: VALORES CATEGÓRICOS VÁLIDOS
print("\n\n4️⃣ VALORES CATEGÓRICOS VÁLIDOS:")
print("-"*80)

categorical_constraints = {
    'Gender': ['Male', 'Female', 'M', 'F', 0, 1],
    'Smoking': [0, 1, 'Yes', 'No'],
    'FamilyHistoryAlzheimers': [0, 1, 'Yes', 'No'],
    'CardiovascularDisease': [0, 1, 'Yes', 'No'],
    'Diabetes': [0, 1, 'Yes', 'No'],
    'Depression': [0, 1, 'Yes', 'No'],
    'HeadInjury': [0, 1, 'Yes', 'No'],
    'Hypertension': [0, 1, 'Yes', 'No']
}

for col, valid_values in categorical_constraints.items():
    if col in df_validate.columns:
        invalid_count = (~df_validate[col].isin(valid_values)).sum()
        unique_invalid = df_validate[~df_validate[col].isin(valid_values)][col].unique()
        
        print(f"\n   • {col}")
        print(f"     Valores válidos: {valid_values}")
        print(f"     Valores inválidos: {invalid_count}")
        
        if invalid_count > 0:
            print(f"     ⚠️ Valores no reconocidos: {unique_invalid}")
        else:
            print(f"     ✓ Todos los valores son válidos")

# REGLA 5: CONSISTENCIA TEMPORAL/LÓGICA
print("\n\n5️⃣ CONSISTENCIA LÓGICA ADICIONAL:")
print("-"*80)

consistency_rules = []

# Edad y diagnóstico de Alzheimer
if 'Age' in df_validate.columns and 'Diagnosis' in df_validate.columns:
    # Alzheimer es más común en mayores de 65
    young_alzheimers = df_validate[(df_validate['Age'] < 50) & (df_validate['Diagnosis'] == 1)]
    
    print(f"\n   • Diagnóstico de Alzheimer en menores de 50 años:")
    print(f"     Casos detectados: {len(young_alzheimers)}")
    
    if len(young_alzheimers) > 0:
        print(f"     ⚠️ Alzheimer de inicio temprano (early-onset) - revisar si es válido")
    else:
        print(f"     ✓ No hay casos de Alzheimer en menores de 50 años")

# MMSE bajo con diagnóstico negativo (inconsistencia)
if 'MMSE' in df_validate.columns and 'Diagnosis' in df_validate.columns:
    low_mmse_no_diagnosis = df_validate[(df_validate['MMSE'] < 20) & (df_validate['Diagnosis'] == 0)]
    
    print(f"\n   • MMSE bajo (<20) sin diagnóstico de Alzheimer:")
    print(f"     Casos detectados: {len(low_mmse_no_diagnosis)}")
    
    if len(low_mmse_no_diagnosis) > 0:
        print(f"     ⚠️ Posible inconsistencia: MMSE bajo sugiere deterioro cognitivo")
    else:
        print(f"     ✓ Consistente: MMSE bajo asociado con diagnóstico positivo")

# RESUMEN EJECUTIVO
print("\n\n" + "="*80)
print("📊 RESUMEN DE VALIDACIÓN:")
print("="*80)

total_rules = len(numeric_ranges) + len(logical_rules) + len(mandatory_fields) + len(categorical_constraints)

print(f"\nTotal de reglas de validación definidas: {total_rules}")
print(f"   • Rangos numéricos: {len(numeric_ranges)}")
print(f"   • Relaciones lógicas: {len(logical_rules)}")
print(f"   • Campos obligatorios: {len(mandatory_fields)}")
print(f"   • Restricciones categóricas: {len(categorical_constraints)}")

print("\n💡 RECOMENDACIONES:")
print("-"*80)
print("""
1. Implementar estas reglas en el pipeline de ingesta de datos
2. Crear tests automáticos que validen cada regla
3. Documentar excepciones válidas (e.g., Alzheimer de inicio temprano)
4. Establecer alertas para violaciones críticas
5. Revisar periódicamente rangos basados en nuevos datos
6. Crear funciones de validación reutilizables
""")

# Guardar reglas para uso posterior
data_validation_rules = {
    'numeric_ranges': numeric_ranges,
    'logical_rules': logical_rules,
    'mandatory_fields': mandatory_fields,
    'categorical_constraints': categorical_constraints
}

print("\n✅ Reglas de validación documentadas y guardadas en 'data_validation_rules'")
print("="*80)


## 9. Resumen Ejecutivo

Conclusiones y recomendaciones generales sobre el dataset.

In [ ]:
print("\n" + "="*80)
print("RESUMEN EJECUTIVO DEL EDA")
print("="*80 + "\n")

print(f"📊 Dataset: {data_path}")
print(f"📈 Dimensiones: {df.shape[0]:,} filas × {df.shape[1]} columnas")
print(f"\n🔍 Composición del Dataset:")
print(f"   • Variables numéricas: {len(numeric_cols)}")
print(f"   • Variables categóricas: {len(categorical_cols)}")

# Calidad de datos
pct_missing = (df.isnull().sum().sum() / (df.shape[0] * df.shape[1]) * 100)
print(f"\n⚙️  Calidad de Datos:")
print(f"   • % Valores faltantes totales: {pct_missing:.2f}%")
print(f"   • Rows completas: {len(df.dropna()):,} ({(len(df.dropna())/len(df)*100):.1f}%)")

# Recomendaciones
print(f"\n💡 Recomendaciones:")

if pct_missing > 5:
    print(f"   ⚠️  Alta proporción de valores faltantes ({pct_missing:.2f}%). Considerar:")
    print(f"       - Imputación de valores")
    print(f"       - Eliminación de columnas/filas con muchos faltantes")
else:
    print(f"   ✓ Proporción aceptable de valores faltantes ({pct_missing:.2f}%)")

if len(outlier_summary) > 0:
    print(f"   ⚠️  Se detectaron outliers. Considerar:")
    print(f"       - Análisis causa-efecto de los outliers")
    print(f"       - Posible transformación o remoción")
else:
    print(f"   ✓ No hay outliers significativos detectados")

if len(numeric_cols) > 1 and strong_corr:
    print(f"   💬 Se encontraron {len(strong_corr)} correlación(es) fuerte(s)")
    print(f"       - Revisar multicolinealidad para modelos predictivos")

print(f"\n✅ EDA completado exitosamente")
print("="*80)